In [1]:
# Imports & config
import os
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
BASE_URL = "https://api.etherscan.io/v2/api"


In [2]:
def fetch_txs(address, action, page=1, offset=10000):
    params = {
        "chainid": 1,
        "module": "account",
        "action": action,
        "address": address,
        "startblock": 0,
        "endblock": 99999999,
        "page": page,
        "offset": offset,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY
    }
    r = requests.get(BASE_URL, params=params)
    r.raise_for_status()
    resp = r.json()
    return resp.get("result", []) if resp.get("status") == "1" else []

def fetch_all(address, action):
    page = 1
    txs = []
    while True:
        batch = fetch_txs(address, action, page=page)
        if not batch:
            break
        txs.extend(batch)
        if len(batch) < 10000:
            break
        page += 1
    return txs


In [ ]:
wallet_df = pd.read_csv(r"C:\Users\kevin\Desktop\VSCode\Zeru\task_2\Wallet id.csv")
wallet_df.head()

,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [4]:
wallet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   wallet_id  103 non-null    object
dtypes: object(1)
memory usage: 956.0+ bytes


In [5]:
wallet_df.isnull().sum()

wallet_id    0
dtype: int64

In [6]:
wallet_df.duplicated().sum()

0

In [7]:
wallets = wallet_df["wallet_id"].tolist()
combined_rows = []

for w in wallets:
    for action, label in [("txlist", "external"), ("txlistinternal", "internal")]:
        txs = fetch_all(w, action)
        for tx in txs:
            combined_rows.append({
                "wallet_id": w,
                "action": label,
                "timestamp": int(tx["timeStamp"]),
                "value": float(tx.get("value", 0)) / 1e18,
                "gas": int(tx.get("gas", 0)),
                "gasUsed": int(tx.get("gasUsed", 0))
            })

df = pd.DataFrame(combined_rows)

In [ ]:
# Save final CSV
output_dir = r"C:\Users\kevin\Desktop\VSCode\Zeru\task_2\output"
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir, "wallet_txns_combined.csv"), index=False)